In [1]:
import requests
from bs4 import BeautifulSoup

In [8]:
url = 'https://www.latam.com/es_co/apps/personas/booking?fecha1_dia=31&fecha1_anomes=2020-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Bogot%C3%A1&from_city1=BOG&vuelos_destino=Ciudad%20de%20M%C3%A9xico&to_city1=MEX&flex=1&vuelos_fecha_salida_ddmmaaaa=11/09/2020&cabina=Y&nadults=2&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [3]:
# To handle 403 error
#headers = {'User-Agent': 'Chrome/85.0.4183.83'}
#r = requests.get(url, headers=headers)

In [4]:
#r.status_code

In [5]:
#s = BeautifulSoup(r.text,'lxml')

In [6]:
# print(s.prettify())

## Importando selenium

# XPATH CheatSheet

http://labs.timtom.ch/library-webscraping/extras/xpath-cheatsheet.md.pdf

In [6]:
from selenium import webdriver

## 1. Instanciando el driver de Chrome

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)

## 2. Cargar la url en el navegador

In [5]:
driver.get(url)

## 3. Extraer la información de la página

In [49]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="0f82d6ce-51e9-40a0-bb22-11dd479d057a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="838629da-ced3-479a-9473-66b82054120a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="5be4c3e7-e78a-4f73-8631-27b1775ba986")>,
 <selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="a8630aeb-b9ff-459d-a45c-42933df603b7")>]

In [50]:
vuelo = vuelos[0]
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="0f82d6ce-51e9-40a0-bb22-11dd479d057a")>

### Hora de salida primer vuelo de la lista

In [51]:
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'18:22'

### >>> Reto: 

Colocar la hora de llegada y la duración del vuelo

### Hora de llegada primer vuelo

In [52]:
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'14:35'

### Duración del vuelo

In [53]:
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT21H13M'

In [54]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
# Abrir el botón
boton_escalas.click()

In [55]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="097b1873-ea2c-4058-820c-74149119eb13")>,
 <selenium.webdriver.remote.webelement.WebElement (session="54e0a554cbf4c5220beeb8373a222558", element="2748ea30-7665-490f-bef9-fb6d3e063207")>]

In [56]:
escalas = len(segmentos) - 1
escalas

1

In [57]:
segmento = segmentos[0]

In [58]:
# Aeropuerto
segmento.find_element_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text

'El Dorado Intl.'

In [59]:
# Hora de salida del primer vuelo
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'18:22'

---

### >>> Reto: 

1. Destino de cada escala
2. Hora de llegada de cada escala
3. Duración de cada vuelo
4. Duración de la escala
5. Número de vuelo
6. Modelo del avión

In [69]:
destinos = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/abbr')
print('>>> Destinos de cada escala:')
#[destino.text for destino in destinos]
destinos[-1].text

>>> Destinos de cada escala:


'MEX'

In [72]:
horas_llegada_por_escala = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/time')
print('>>> Hora llegada por cada escala:')
[hora.get_attribute('datetime') for hora in horas_llegada_por_escala]

>>> Hora llegada por cada escala:


['18:22', '21:25', '09:25', '14:35']

In [20]:
duracion_por_vuelo = segmento.find_elements_by_xpath('//span[@class="sc-esjQYD dMquDU"]/time')
print('>>> Duracion por vuelo:')
duraciones = [duracion.get_attribute('datetime') for duracion in duracion_por_vuelo]
duraciones_vuelos = [duraciones[0] , duraciones[-1]]
duraciones_vuelos


>>> Duracion por vuelo:


['3:3', '6:10']

In [71]:
duraciones

['3:3', '12:0', '6:10']

In [21]:
duracion_escalas = segmento.find_elements_by_xpath('//span[@class="sc-esjQYD dMquDU"]/time')
print('>>> Duracion por escala:')
[duracion.get_attribute('datetime') for duracion in duracion_escalas][1]

>>> Duracion por escala:


'12:0'

In [22]:
numeros_vuelos = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/b')
print('>>> Número de vuelo:')
[numero_vuelo.text for numero_vuelo in numeros_vuelos]

>>> Número de vuelo:


['LA2447', 'LA2472']

In [23]:
modelo_aviones = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]')
print('>>> Modelo Aviones:')
[modelo_avion.text for modelo_avion in modelo_aviones]

>>> Modelo Aviones:


['Airbus 320-200', 'Boeing 767-300']

### Cerrar el modal

In [24]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

### Abrir precios

In [25]:
vuelo.click()

In [38]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class,"fare-")]')

In [44]:
tarifas[0].find_element_by_xpath('.//label').get_attribute('for')

'LIGHT'

---



### >>> Reto

1. Extraer el símbolo de la moneda
2. Extraer el precio de cada categoría

In [45]:
precios = []

for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
    dict_tarifa = {nombre:{'moneda':moneda, 'valor': valor}}
    precios.append(dict_tarifa)

print(precios)

[{'LIGHT': {'moneda': '$', 'valor': '1.657.600'}}, {'PLUS': {'moneda': '$', 'valor': '1.982.160'}}, {'TOP': {'moneda': '$', 'valor': '2.176.830'}}]


In [33]:
precios = [tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text for tarifa in tarifas]
precios

['1.597.200', '1.909.260', '2.096.470']

---

## Creando funciones para automatizar el proceso

In [1]:
def obtener_vuelos(vuelo):
    """
    La función recibe un vuelo y retorna una lista de diccionarios con las distintas tarifas.
    """
    tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class,"fare-")]')
    precios = []

    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
        dict_tarifa = {nombre:{'moneda':moneda, 'valor': valor}}
        precios.append(dict_tarifa)
        
    return precios

In [2]:
def obtener_datos_escalas(vuelo):
    """
    La función retorna una lista de diccionarios con la información de las escalas de cada vuelo.
    """
    segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
    info_escalas = []
    for segmento in segmentos:
        #Origen
        origen = segmento.find_element_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text
        # Hora de salida del primer vuelo
        hora_salida = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')
        destino = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/abbr')[-1].text
        hora_llegada = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/time')[-1].text
        numeros_vuelos = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/b')[0].text
        modelo_avion = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]')[0].text
        if segmento != segmentos[-1]:
            duracion_escalas = segmento.find_elements_by_xpath('//span[@class="sc-esjQYD dMquDU"]/time')
            duracion_escala = [duracion.get_attribute('datetime') for duracion in duracion_escalas][1]
        else:
            duracion_escala = ''
        
        data_dict= {
            'origen':origen,
            'hora_salida':hora_salida,
            'destino':destino,
            'hora_llegada':hora_llegada,
            'numeros_vuelos':numeros_vuelos,
            'modelo_avion':modelo_avion,
            'duracion_escala':duracion_escala   
        }
        info_escalas.append(data_dict)
        
    return info_escalas

In [3]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    
    # Hora de llegada
    llegada =  vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    
    tiempos = {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}
    return tiempos

In [4]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('>>> Iniciando scraping...')
    info=[]
    for vuelo in vuelos:
        tiempos = obtener_tiempos(vuelo)
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        escalas = obtener_datos_escalas(vuelo)
        driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()
        vuelo.click()
        precios = obtener_vuelos(vuelo)
        vuelo.click()
        info.append({
                     'precios':precios,
                     'tiempos':tiempos,
                     'escalas':escalas
                    })
    return info

# Scraper finalizado

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)
driver.get(url)
# Introducir demoras dinámicas
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

delay = 40
try:
    vuelo = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La página terminó de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')

driver.close()


La página terminó de cargar
Se encontraron 4 vuelos.
>>> Iniciando scraping...


In [12]:
#info_vuelos

In [83]:
# obtener_info(driver)

## 4. Cerrar el navegador

In [75]:
# driver.close()